In [1]:
import argparse
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from importlib import import_module

import torch
from dataset import CustomDataLoader, collate_fn, test_transform
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
output_dir = './output'

# load sample_submission.csv
submission = pd.read_csv('../baseline_code/submission/sample_submission.csv', index_col=None)

In [3]:
print("Start prediction..")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

test_dataset = CustomDataLoader(data_dir='/opt/ml/segmentation/input/data/test.json', mode='test', transform=test_transform)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=8,
    num_workers=4,
    shuffle=False,
    pin_memory=use_cuda,
    collate_fn=collate_fn
)

Start prediction..
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


tta augmentation 효과 변경

In [4]:
import ttach as tta
transforms = tta.Compose(
[
    tta.HorizontalFlip(),
    tta.Rotate90(angles=[0, 180]),
    # tta.Resize(sizes=(512,512)),
    # tta.Scale(scales=[1, 2, 4]),
    tta.Multiply(factors=[0.9, 1, 1.1]),      
])

1. 아래 모델명과 가중치 부분을 일치하여 변경해준다

In [9]:
models_name = ['UNet_PlusPlus', 'DeepLabV3_Plus']
ckpt_list = ['/opt/ml/segmentation/semantic-segmentation-level2-cv-06/runs/UNet_PlusPlus2/best.pt',
            '/opt/ml/segmentation/semantic-segmentation-level2-cv-06/runs/deeplab_v3_b7_best.pt']

model1 = torch.load(ckpt_list[0])
model1.to(device)

model2 = torch.load(ckpt_list[1])
model2.to(device)

# model3 = torch.load(ckpt_list[2])
# model3.to(device)

tta_model1 = tta.SegmentationTTAWrapper(model1, transforms)
tta_model2 = tta.SegmentationTTAWrapper(model2, transforms)
# tta_model3 = tta.SegmentationTTAWrapper(model3, transforms)

model1.eval()
tta_model1.eval()
model2.eval()
tta_model2.eval()
# model3.eval()
# tta_model3.eval()

SegmentationTTAWrapper(
  (model): DeepLabV3_Plus(
    (model): DeepLabV3Plus(
      (encoder): EfficientNetEncoder(
        (_conv_stem): Conv2dStaticSamePadding(
          3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
          (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
        )
        (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_blocks): ModuleList(
          (0): MBConvBlock(
            (_depthwise_conv): Conv2dStaticSamePadding(
              64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
              (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
            )
            (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
            (_se_reduce): Conv2dStaticSamePadding(
              64, 16, kernel_size=(1, 1), stride=(1, 1)
              (static_padding): Identity()
            )
            (_s

In [10]:
size = 256
transform = A.Compose([A.Resize(size, size)])

file_name_list = []
preds_array = np.empty((0, size*size), dtype=np.compat.long)

In [12]:
with torch.no_grad():
    for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
        # inference (512 x 512)
        outs = 0
        for model_name in models_name:
            if model_name in ('FCNRes50', 'FCNRes101', 'DeepLabV3_Res50', 'DeepLabV3_Res101'):
                # output size : ([8, 11, 512, 512])
                outs += model1(torch.stack(imgs).to(device))['out']
                outs += tta_model1(torch.stack(imgs).to(device))['out']
                # print('if : ', model_name)
            else:
                # 2. 원하는 모델을 더해주고 더해준 모델의 개수만큼 나눠준다.
                images = torch.stack(imgs).to(device)

                outs += model1(images)
                outs += tta_model1(images)

                outs += model2(images)
                outs += tta_model2(images)

                # outs += model3(images)
                # outs += tta_model3(images)
                
                # print('else : ', model_name)
                # outs = (1/4)*(model1(images)+tta_model1(images)+model2(images)+tta_model2(images))   
                
            outs = (1/4)*outs ### 모델 개수만큼 바꿔줘야 됨!

        oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy() # (8, 512, 512)

        # resize (256 x 256)
        temp_mask = []
        for img, mask in zip(np.stack(imgs), oms):
            transformed = transform(image=img, mask=mask)
            mask = transformed['mask']
            temp_mask.append(mask)


        oms = np.array(temp_mask)
        oms = oms.reshape([oms.shape[0], size*size]).astype(int)
        preds_array = np.vstack((preds_array, oms))

        file_name_list.append([i['file_name'] for i in image_infos])

print("End prediction!")
file_names = [y for x in file_name_list for y in x]

  2%|▏         | 2/103 [00:31<26:40, 15.85s/it]

KeyboardInterrupt: 

In [14]:
# write PredictionString
for file_name, string in zip(file_names, preds_array):
    submission = submission.append({"image_id": file_name, "PredictionString": ' '.join(str(e) for e in string.tolist())},
                                    ignore_index=True)

# save submission.csv
submission.to_csv(output_dir+'/submission_unetplus_deeplabv3.csv', index=False)